# Color Analysis

In [ ]:
from analyze import analyze_color_histogram
from scg_detection_tools.models import YOLO_NAS, YOLOv8
from scg_detection_tools.detect import Detector
from scg_detection_tools.utils.file_handling import read_yaml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import cv2

cfg = read_yaml("analyze_config.yaml")
#model = YOLOv8(yolov8_ckpt_path=cfg["yolov8_model_path"])
model = YOLO_NAS(model_arch=cfg["yolonas_arch"], 
                 checkpoint_path=cfg["yolonas_model_path"], 
                 classes=cfg["data_classes"])
det_params = cfg["detect_parameters"]
det_params["embed_slice_callback"] = None
det = Detector(detection_model=model, detection_params=det_params)


In [ ]:
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

IMG_DIR = "/home/julio/Dev/SCG_IFSC/save/light_group_annotations/images"
LBL_DIR = "/home/julio/Dev/SCG_IFSC/save/light_group_annotations/labels"

imgs = [
    #f"{IMG_DIR}/0_640x640_png.rf.e445f1e2ec7d5f09191bbfac000f8274.jpg",
    #f"{IMG_DIR}/1_640x640_png.rf.c186ca44871ae1749eec239fad71fa84.jpg",
    #f"{IMG_DIR}/3_640x640_sparse_png.rf.4be15566a8c7a2ba76e1fc057f0755a3.jpg",
    #f"{IMG_DIR}/e6_png.rf.517fd5fb9cbeb985254297ed7bad1d86.jpg",
    f"{IMG_DIR}/e4.png"
]
#dets = det(imgs[0])[0]
IMG_DIR = "/home/julio/Downloads/train/images"
imgs = get_all_files_from_paths(IMG_DIR)
LBL_DIR = "/home/julio/Downloads/train/labels"
img_hists = analyze_color_histogram(model=model, detector=det, imgs=imgs, raw=False, on_detection_boxes=False, seg_annotations=LBL_DIR, cspaces=["RGB", "HSV", "GRAY"], show=True)


In [ ]:
cspaceAnalysis = {"RGB": [], "HSV": [], "GRAY": [], "ALL": []}
nclass = {0: "light", 1: "medium", 2: "dark", 3: "dead"}
intensities = np.arange(256)
for img in img_hists:
    img_hist = img_hists[img]
    for cspace in ["RGB", "HSV", "GRAY"]:
        for mask_hist in img_hist[cspace]["masks"]:
            hist = mask_hist["hist"]
            mask_class = nclass[mask_hist["class"]]

            ch_stats = []
            for ch_hist in hist:
                ch_mean = np.sum(intensities * ch_hist) / np.sum(ch_hist)
                ch_std = np.sqrt(np.sum((intensities - ch_mean) ** 2 * ch_hist) / np.sum(ch_hist))
                ch_stats.append([ch_mean, ch_std])
    
                all_mean[-1].append(ch_mean)
                all_std[-1].append(ch_std)
            
            ch_stats = np.array(ch_stats).T.ravel().tolist()
            ch_stats.append(mask_class)
            print(f"mask hists: {cspace} {ch_stats}")
            cspaceAnalysis[cspace].append(ch_stats)

num_masks = len(cspaceAnalysis["RGB"])
all_mean = []
all_std = []
all_class = [data[-1] for data in cspaceAnalysis["RGB"]]

for maskidx in range(num_masks):
    all_mean.append([])
    all_std.append([])
    for cspace in ["RGB", "HSV", "GRAY"]:
        data = cspaceAnalysis[cspace][maskidx]
        print(f"data for mask {maskidx}: {data}")
        if cspace == "GRAY":
            ch_mean = data[0]
            ch_std = data[1]
            print(f"GRAY: maskidx={maskidx}, ch_mean={ch_mean}, ch_std={ch_std}")
            all_mean[-1].append(ch_mean)
            all_std[-1].append(ch_std)
        else:
            ch_mean = data[:3]
            ch_std = data[3:-1]
            print(f"RGB or HSV: maskidx={maskidx}, ch_mean={ch_mean}, ch_std={ch_std}")
            all_mean[-1].extend(ch_mean)
            all_std[-1].extend(ch_std)
for i in range(num_masks):
    cspaceAnalysis["ALL"].append([])
    
    cspaceAnalysis["ALL"][i].extend(all_mean[i])
    cspaceAnalysis["ALL"][i].extend(all_std[i])
    cspaceAnalysis["ALL"][i].append(all_class[i])

rgbdf = pd.DataFrame(cspaceAnalysis["RGB"], columns=[
    "R Mean", "G Mean", "B Mean", "R std", "G std", "B std", "Class"
])
hsvdf = pd.DataFrame(cspaceAnalysis["HSV"], columns=[
    "H Mean", "S Mean", "V Mean", "H std", "S std", "V std", "Class"
])
graydf = pd.DataFrame(cspaceAnalysis["GRAY"], columns=[
    "Gray mean", "Gray std", "Class"
])

alldf = pd.DataFrame(cspaceAnalysis["ALL"], columns=[
    "R Mean", "G Mean", "B Mean", "H Mean", "S Mean", "V Mean", "Gray mean", "R std", "G std", "B std", "H std", "S std", "V std", "Gray std", "Class"
])

class_color = lambda df: df["Class"].map({
    "light": "b", 
    "medium": "g", 
    "dark": "r", 
    "dead": "k"
})

scatter_matrix(rgbdf.iloc[:,:-1], figsize=(12,12), diagonal="kde", c=class_color(rgbdf))
scatter_matrix(hsvdf.iloc[:,:-1], figsize=(12,12), diagonal="kde", c=class_color(hsvdf))
scatter_matrix(graydf.iloc[:,:-1], figsize=(12,12), diagonal="kde", c=class_color(graydf))
scatter_matrix(alldf.iloc[:,:-1], figsize=(12,12), diagonal="kde", c=class_color(alldf))

print(rgbdf.corr(numeric_only=True))
print(hsvdf.corr(numeric_only=True))
print(graydf.corr(numeric_only=True))

In [ ]:
print(len(img_hists[imgs[0]]["RGB"]["masks"]))

# Pixel density

In [ ]:
from analyze import analyze_pixel_density

IMG_DIR = "/home/julio/Dev/SCG_IFSC/scg-leaf-count/imgs/test"
LBL_DIR = "/home/julio/Dev/SCG_IFSC/scg-leaf-count/imgs/testlabels"

imgs = [
    f"{IMG_DIR}/e1.png",
    f"{IMG_DIR}/e3.png",
    f"{IMG_DIR}/e4.png",
]

densities = analyze_pixel_density(model=model, 
                                  detector=det, 
                                  imgs=imgs, 
                                  cfg=cfg, 
                                  on_slice=False, 
                                  on_detection_boxes=False,
                                  on_crops=True,
                                  seg_annotations=None, 
                                  save_detections=False)
print(densities)